In [24]:
import guidance
from llama_quantized import LLaMAQuantized
from search import web_search, find_closest_sections

In [25]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the MODEL_PATH and MODEL_NAME from environment variables
model_dir = os.getenv("MODEL_PATH")
model = os.getenv("MODEL_NAME")

# Initialize the LLaMAQuantized model
#llama_model = LLaMAQuantized(model_dir=model_dir, model=model)
guidance.llm = LLaMAQuantized(model_dir=model_dir, model=model)

Found the following quantized model: /home/pwalch/projects/textgen/models/4bit_WizardLM-13B-Uncensored-4bit-128g/4bit-128g.safetensors


2023-09-04 02:44:21,648 - 139843890763584 - base_events.py-base_events:1758 - ERROR: Task exception was never retrieved
future: <Task finished name='Task-34' coro=<Program.execute() done, defined at /home/pwalch/miniconda3/envs/tg/lib/python3.10/site-packages/guidance/_program.py:430> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/pwalch/miniconda3/envs/tg/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_144221/1430233959.py", line 12, in <module>
    executed_program = program(
  File "/home/pwalch/miniconda3/envs/tg/lib/python3.10/site-packages/guidance/_program.py", line 295, in __call__
    loop.run_until_complete(new_program.execute())
  File "/home/pwalch/miniconda3/envs/tg/lib/python3.10/site-packages/nest_asyncio.py", line 93, in run_until_complete
    self._run_once()
  File "/home/pwalch/miniconda3/envs/tg/lib/python3.10/site-pack

Model to device


In [22]:
del guidance.llm 

In [23]:
guidance.llm.cache.clear()
print('Done.')

AttributeError: module 'guidance' has no attribute 'llm'

In [21]:
valid_answers = ['Action', 'Final Answer']
valid_tools = ['Google Search']

prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following questions as best you can. You have access to the following tools:

Google Search: A wrapper around Google Search. Useful for when you need to answer questions about current events. The input is the question to search relavant information.

Strictly use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Google Search]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For examples:
Question: How old is CEO of Microsoft wife?
Thought: First, I need to find who is the CEO of Microsoft.
Action: Google Search
Action Input: Who is the CEO of Microsoft?
Observation: Satya Nadella is the CEO of Microsoft.
Thought: Now, I should find out Satya Nadella's wife.
Action: Google Search
Action Input: Who is Satya Nadella's wife?
Observation: Satya Nadella's wife's name is Anupama Nadella.
Thought: Then, I need to check Anupama Nadella's age.
Action: Google Search
Action Input: How old is Anupama Nadella?
Observation: Anupama Nadella's age is 50.
Thought: I now know the final answer.
Final Answer: Anupama Nadella is 50 years old.

### Input:
{{question}}

### Response:
Question: {{question}}
Thought: {{gen 'thought' stop='\\n'}}
Action: {{select 'tool_name' options=valid_tools}}
Action Input: {{gen 'actInput' stop='\\n'}}
Observation:{{search actInput}}
Thought: {{gen 'thought2' stop='\\n'}}
Final Answer: {{gen 'final' stop='\\n'}}"""

def searchGoogle(t):    
    return search.run(t)

prompt = guidance(prompt_template)
result = prompt(question='Is Eminem a football player?', search=searchGoogle, valid_answers=valid_answers, valid_tools=valid_tools)


Stop program Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following questions as best you can. You have access to the following tools:

Google Search: A wrapper around Google Search. Useful for when you need to answer questions about current events. The input is the question to search relavant information.

Strictly use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Google Search]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For examples:
Question: How old is CEO of Microsoft wife?
Thought: First, I need to find who is the CEO of Microsoft.
Action: Google Search
Action Input: Who is the CEO of Microsoft?
Observation: Satya Nadella is the CEO of Microsoft.
Thought: Now, I should find out Satya Nadella's wife.
Action: Google Search
Action Input: Who is Satya Nadella's wife?
Observation: Satya Nadella's wife's name is Anupama Nadella.
Thought: Then, I need to check Anupama Nadella's age.
Action: Google Search
Action Input: How old is Anupama Nadella?
Observation: Anupama Nadella's age is 50.
Thought: I now know the final answer.
Final Answer: Anupama Nadella is 50 years old.

### Input:
 {{question}} 

### Response:
Question: {{question}} 
Thought: {{gen 'thought' stop='\n'}} 
Action: {{select 'tool_name' options=valid_tools}} 
Action Input: {{gen 'actInput' stop='\n'}} 
Observation: {{search actInput}} 
Thought: {{gen 'thought2' stop='\n'}} 
Final Answer: {{gen 'final' stop='\n'}}

KeyboardInterrupt: 

In [11]:
def searchGoogle(key_word):    
    return search.run(key_word)

dict_tools = {
    'Google Search': searchGoogle
}

In [12]:
class CustomAgentGuidance:
    def __init__(self, guidance, tools, num_iter=3):
        self.guidance = guidance
        self.tools = tools
        self.num_iter = num_iter

    def do_tool(self, tool_name, actInput):
        return self.tools[tool_name](actInput)
    
    def __call__(self, query):
        prompt_start = self.guidance(prompt_start_template)
        result_start = prompt_start(question=query, valid_answers=valid_answers)

        result_mid = result_start
        
        for _ in range(self.num_iter - 1):
            if result_mid['answer'] == 'Final Answer':
                break
            history = result_mid.__str__()
            prompt_mid = self.guidance(prompt_mid_template)
            result_mid = prompt_mid(history=history, do_tool=self.do_tool, valid_answers=valid_answers, valid_tools=valid_tools)
        
        if result_mid['answer'] != 'Final Answer':
            history = result_mid.__str__()
            prompt_mid = self.guidance(prompt_final_template)
            result_final = prompt_mid(history=history, do_tool=self.do_tool, valid_answers=['Final Answer'], valid_tools=valid_tools)
        else:
            history = result_mid.__str__()
            prompt_mid = self.guidance(history + "{{gen 'fn' stop='\\n'}}")
            result_final = prompt_mid()
        return result_final['fn']

In [19]:
custom_agent = CustomAgentGuidance(guidance, dict_tools)

list_queries = [
    "How much is the salary of number 8 of Manchester United?",
    "What is the population of Congo?",
    "Where was the first president of South Korean born?",
    "What is the population of the country that won World Cup 2022?"    
]

final_answer = custom_agent(list_queries[0])

NameError: name 'prompt_start_template' is not defined

In [20]:
# define a guidance program that adapts a proverb
program = guidance("""Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{#select 'chapter'}}9{{or}}10{{or}}11{{/select}}:{{gen 'verse'}}""")

# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)
print(executed_program)

Stop program Tweak this proverb to apply to model instructions instead.

 {{proverb}} 
- {{book}} {{chapter}} : {{verse}} 

UPDATED
Where there is no guidance {{gen 'rewrite' stop="\n-"}} 
- GPT {{#select 'chapter'}} 9 {{or}} 10 {{or}} 11 {{/select}} : {{gen 'verse'}}

KeyboardInterrupt: 

In [14]:
demo_results = [{'text': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'text': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'text': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]'}]
practice_round = guidance(
'''{{#user~}}
Who are the founders of OpenAI?
{{~/user}}
{{#assistant~}}
<search>OpenAI founders</search>
{{~/assistant}}
{{#user~}}
Search results:
{{~#each results}}
<result>
{{this.text}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}''', guidance.llm)
practice_round = practice_round(results=demo_results)

def is_search(completion):
    return '<search>' in completion

def search(query):
    search_results = web_search(query)
    scored_results = find_closest_sections(search_results, query)
    scored_results = scored_results[:4]
    print(scored_results)
    return scored_results

prompt = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. But please respond as if I have not seen the results.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round.
{{~/assistant}}
{{>practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.text}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''', model)

prompt = prompt(practice_round=practice_round, search=search, is_search=is_search, user_query="Where are the best places to see cherry blossoms in Seattle?")

Stop program {{#system~}} 
You are a helpful assistant.
 {{~/system}} 
 {{#user~}} 
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. But please respond as if I have not seen the results.
 {{~/user}} 
 {{#assistant~}} 
Ok, I will do that. Let's do a practice round.
 {{~/assistant}} 
 {{>practice_round}} 
 {{#user~}} 
That was great, now let's do another one.
 {{~/user}} 
 {{#assistant~}} 
Ok, I'm ready.
 {{~/assistant}} 
 {{#user~}} 
 {{user_query}} 
 {{~/user}} 
 {{#assistant~}} 
 {{gen "query" stop="</search>"}} {{#if (is_search query)}} </search> {{/if}} 
 {{~/assistant}} 
 {{#if (is_search query)}} 
 {{#user~}} 
Search results: {{#each (search query)}} 
<result>
 {{this.text}} 
</result> {{/each}} 
 {{~/user}} 
 {{#assistant~}} 
 {{gen "answer"}} 
 {{~/assistant}} 
 {{/if}}